## Run the Whole Pipeline

In [1]:
from op_analytics.datasources.chainsmeta.systemconfig.execute import execute_pull

result = execute_pull()

2025-06-13 10:17:02 [info     ] initialized duckdb at /var/folders/tz/g2vwn0qx5ll32j7jfsshhps40000gn/T/gdpgb6gk/op-analytics.duck.db filename=client.py lineno=118 process=30242
2025-06-13 10:17:02 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=30242
2025-06-13 10:17:02 [debug    ] loaded vault: 21 items         filename=vault.py lineno=79 process=30242
2025-06-13 10:17:02 [info     ] create duckddb gcs secret      filename=client.py lineno=66 process=30242
2025-06-13 10:17:02 [info     ] Reading data from 'chainsmeta/superchain_chain_list_v1' with filters min_date=None, max_date=None, date_range_spec=None filename=dailydata.py lineno=94 process=30242
2025-06-13 10:17:04 [info     ] registered view: 'chainsmeta_superchain_chain_list_v1' using uri wildcard 'gs://oplabs-tools-data-sink/chainsmeta/superchain_chain_list_v1/dt=*/out.parquet' filename=client.py lineno=319 process=30242
┌─────────────────────────────────────┐
│                name               

Exception: 
        Mismatching schema:
        
        ["name  String", "identifier  String", "chain_id  Int64", "rpc_url  String", "system_config_proxy  String", "block_number  Int64", "block_timestamp  Int64", "batch_inbox_slot  String", "dispute_game_factory_slot  String", "l1_cross_domain_messenger_slot  String", "l1_erc721_bridge_slot  String", "l1_standard_bridge_slot  String", "optimism_mintable_erc20_factory_slot  String", "optimism_portal_slot  String", "start_block_slot  String", "unsafe_block_signer_slot  String", "version  String", "basefee_scalar  Int64", "batch_inbox  String", "batcher_hash  String", "blob_basefee_scalar  Int64", "dispute_game_factory  String", "eip1559_denominator  Int64", "eip1559_elasticity  Int64", "gas_limit  Int64", "l1_cross_domain_messenger  String", "l1_erc721_bridge  String", "l1_standard_bridge  String", "maximum_gas_limit  Int64", "minimum_gas_limit  Int64", "operator_fee_constant  Int64", "operator_fee_scalar  Int64", "optimism_mintable_erc

In [2]:
result.head()

In [ ]:
import polars as pl

# check if any null values
result.filter(pl.any_horizontal(pl.all().is_null())).head()

# List of chains to fetch

In [1]:
from op_analytics.datasources.chainsmeta.systemconfig.chainslist import ChainsList

chains =ChainsList.fetch()
chains

2025-06-13 10:02:11 [info     ] initialized duckdb at /var/folders/tz/g2vwn0qx5ll32j7jfsshhps40000gn/T/5bhaf159/op-analytics.duck.db filename=client.py lineno=118 process=28065
2025-06-13 10:02:11 [info     ] loaded vault from .env file    filename=vault.py lineno=32 process=28065
2025-06-13 10:02:11 [debug    ] loaded vault: 21 items         filename=vault.py lineno=79 process=28065
2025-06-13 10:02:11 [info     ] create duckddb gcs secret      filename=client.py lineno=66 process=28065
2025-06-13 10:02:11 [info     ] Reading data from 'chainsmeta/superchain_chain_list_v1' with filters min_date=None, max_date=None, date_range_spec=None filename=dailydata.py lineno=94 process=28065
2025-06-13 10:02:14 [info     ] registered view: 'chainsmeta_superchain_chain_list_v1' using uri wildcard 'gs://oplabs-tools-data-sink/chainsmeta/superchain_chain_list_v1/dt=*/out.parquet' filename=client.py lineno=319 process=28065
┌─────────────────────────────────────┐
│                name               

ChainsList(data=shape: (34, 5)
┌────────────────────┬──────────────────────┬──────────┬─────────────────────┬─────────────────────┐
│ name               ┆ identifier           ┆ chain_id ┆ rpc_url             ┆ system_config_proxy │
│ ---                ┆ ---                  ┆ ---      ┆ ---                 ┆ ---                 │
│ str                ┆ str                  ┆ i32      ┆ str                 ┆ str                 │
╞════════════════════╪══════════════════════╪══════════╪═════════════════════╪═════════════════════╡
│ OP Mainnet         ┆ mainnet/op           ┆ 10       ┆ https://mainnet.opt ┆ 0x229047fed2591dbec │
│                    ┆                      ┆          ┆ imism.io            ┆ 1ef1118d64f…        │
│ Unichain           ┆ mainnet/unichain     ┆ 130      ┆ https://mainnet.uni ┆ 0xc407398d063f942fe │
│                    ┆                      ┆          ┆ chain.org           ┆ bbcc6f80a15…        │
│ HashKey Chain      ┆ mainnet/hashkeychain ┆ 177      ┆ htt

In [2]:
chains.data.to_dicts()[0]

{'name': 'OP Mainnet',
 'identifier': 'mainnet/op',
 'chain_id': 10,
 'rpc_url': 'https://mainnet.optimism.io',
 'system_config_proxy': '0x229047fed2591dbec1ef1118d64f7af3db9eb290'}

## Single Chain Debugging

In [ ]:
from op_analytics.datasources.chainsmeta.systemconfig.chain import ChainSystemConfig


system_config = ChainSystemConfig.fetch(
    chain_id=10,
    system_config_proxy='0x229047fed2591dbec1ef1118d64f7af3db9eb290'
)

system_config.data

2025-06-13 10:02:19 [info     ] fetched system config metadata from rpc for chain 10 filename=chain.py lineno=58 process=28065


{'contract_address': '0x229047fed2591dbec1ef1118d64f7af3db9eb290',
 'block_number': 22696105,
 'block_timestamp': 1749823331,
 'batch_inbox_slot': '0x71ac12829d66ee73d8d95bff50b3589745ce57edae70a3fb111a2342464dc597',
 'dispute_game_factory_slot': '0x52322a25d9f59ea17656545543306b7aef62bc0cc53a0e65ccfa0c75b97aa906',
 'l1_cross_domain_messenger_slot': '0x383f291819e6d54073bc9a648251d97421076bdd101933c0c022219ce9580636',
 'l1_erc721_bridge_slot': '0x46adcbebc6be8ce551740c29c47c8798210f23f7f4086c41752944352568d5a7',
 'l1_standard_bridge_slot': '0x9904ba90dde5696cda05c9e0dab5cbaa0fea005ace4d11218a02ac668dad6376',
 'optimism_mintable_erc20_factory_slot': '0xa04c5bb938ca6fc46d95553abf0a76345ce3e722a30bf4f74928b8e7d852320c',
 'optimism_portal_slot': '0x4b6c74f9e688cb39801f2112c14a8c57232a3fc5202e1444126d4bce86eb19ac',
 'start_block_slot': '0xa11ee3ab75b40e88a0105e935d17cd36c8faee0138320d776c411291bdbbb19f',
 'unsafe_block_signer_slot': '0x65a7ed542fb37fe237fdfbdd70b31598523fe5b32879e307bae27a0

# Multi-Chain Debugging

In [4]:
from op_analytics.datasources.chainsmeta.systemconfig.chains import ManyChainsSystemConfig


full_config = ManyChainsSystemConfig.fetch(chains)

2025-06-13 10:02:19 [debug    ] method eip1559Denominator failed with -32000 for RPCManager(system_config_proxy='0x43f8defe3e9286d152e91bb16a248808e7247198') counter=003/034 eta=None filename=rpc.py lineno=199 process=28065
2025-06-13 10:02:19 [debug    ] method eip1559Elasticity failed with -32000 for RPCManager(system_config_proxy='0x43f8defe3e9286d152e91bb16a248808e7247198') counter=003/034 eta=None filename=rpc.py lineno=199 process=28065
2025-06-13 10:02:19 [debug    ] method operatorFeeConstant failed with -32000 for RPCManager(system_config_proxy='0x43f8defe3e9286d152e91bb16a248808e7247198') counter=003/034 eta=None filename=rpc.py lineno=199 process=28065
2025-06-13 10:02:19 [debug    ] method operatorFeeScalar failed with -32000 for RPCManager(system_config_proxy='0x43f8defe3e9286d152e91bb16a248808e7247198') counter=003/034 eta=None filename=rpc.py lineno=199 process=28065
2025-06-13 10:02:19 [info     ] methods ['eip1559Denominator', 'eip1559Elasticity', 'operatorFeeConstant'

In [5]:
full_config.data

contract_address,block_number,block_timestamp,batch_inbox_slot,dispute_game_factory_slot,l1_cross_domain_messenger_slot,l1_erc721_bridge_slot,l1_standard_bridge_slot,optimism_mintable_erc20_factory_slot,optimism_portal_slot,start_block_slot,unsafe_block_signer_slot,version,basefee_scalar,batch_inbox,batcher_hash,blob_basefee_scalar,dispute_game_factory,eip1559_denominator,eip1559_elasticity,gas_limit,l1_cross_domain_messenger,l1_erc721_bridge,l1_standard_bridge,maximum_gas_limit,minimum_gas_limit,operator_fee_constant,operator_fee_scalar,optimism_mintable_erc20_factory,optimism_portal,overhead,owner,scalar,start_block,unsafe_block_signer,version_hex,chain_id,system_config_proxy
str,i64,i64,str,str,str,str,str,str,str,str,str,str,i64,str,str,i64,str,i64,i64,i64,str,str,str,i64,i64,i64,i64,str,str,str,str,str,str,str,null,i64,str
"""0x229047fed2591dbec1ef1118d64f…",22696105,1749823331,"""0x71ac12829d66ee73d8d95bff50b3…","""0x52322a25d9f59ea1765654554330…","""0x383f291819e6d54073bc9a648251…","""0x46adcbebc6be8ce551740c29c47c…","""0x9904ba90dde5696cda05c9e0dab5…","""0xa04c5bb938ca6fc46d95553abf0a…","""0x4b6c74f9e688cb39801f2112c14a…","""0xa11ee3ab75b40e88a0105e935d17…","""0x65a7ed542fb37fe237fdfbdd70b3…","""0""",5227,"""0xff00000000000000000000000000…","""0x0000000000000000000000006887…",1014213,"""0xe5965ab5962edc7477c8520243a9…",250,2,40000000,"""0x25ace71c97b33cc4729cf772ae26…","""0x5a7749f83b81b301cab5f48eb851…","""0x99c9fc46f92e8a1c0dec1b1747d0…",200000000,21000000,0,0,"""0x75505a97bd334e7bd3c476893285…","""0xbeb5fc579115071764c7423a4f12…","""0""","""0x847b5c174615b1b7fdf770882256…","""452312848583266388373324160190…","""0x0000000000000000000000000000…","""0xaaaa45d9549eda09e70937013520…",null,10,"""0x229047fed2591dbec1ef1118d64f…"
"""0xc407398d063f942febbcc6f80a15…",22696105,1749823331,"""0x71ac12829d66ee73d8d95bff50b3…","""0x52322a25d9f59ea1765654554330…","""0x383f291819e6d54073bc9a648251…","""0x46adcbebc6be8ce551740c29c47c…","""0x9904ba90dde5696cda05c9e0dab5…","""0xa04c5bb938ca6fc46d95553abf0a…","""0x4b6c74f9e688cb39801f2112c14a…","""0xa11ee3ab75b40e88a0105e935d17…","""0x65a7ed542fb37fe237fdfbdd70b3…","""0""",2000,"""0xff00000000000000000000000000…","""0x0000000000000000000000002f60…",900000,"""0x2f12d621a16e2d3285929c9996f4…",0,0,30000000,"""0x9a3d64e386c18cb1d6d5179a9596…","""0xd04d0d87e0bd4d2e50286760a3ef…","""0x81014f44b0a345033bb2b3b21c7a…",200000000,21000000,0,0,"""0xa2b597eaeacb6f627e088cbead31…","""0x0bd48f6b86a26d3a217d0fa6ffe2…","""2000""","""0x9245d5d10aa8a842b31530de71ea…","""452312848583266388373324160190…","""0x0000000000000000000000000000…","""0x833c6f278474a78658af91ae8edc…",null,130,"""0xc407398d063f942febbcc6f80a15…"
"""0x43f8defe3e9286d152e91bb16a24…",22696105,1749823331,"""0x71ac12829d66ee73d8d95bff50b3…","""0x52322a25d9f59ea1765654554330…","""0x383f291819e6d54073bc9a648251…","""0x46adcbebc6be8ce551740c29c47c…","""0x9904ba90dde5696cda05c9e0dab5…","""0xa04c5bb938ca6fc46d95553abf0a…","""0x4b6c74f9e688cb39801f2112c14a…","""0xa11ee3ab75b40e88a0105e935d17…","""0x65a7ed542fb37fe237fdfbdd70b3…","""0""",20000000,"""0x0004cb44c80b6fbf8ceb1d80af68…","""0x000000000000000000000000f09e…",4294967295,"""0x04ec030f362ce5a0b5fe2d4b4219…",null,null,30000000,"""0x899f07862d3a03f70e07b7f01183…","""0xd4c83d93c6fae3e0804b785f9cf4…","""0x2171e6d3b7964fa9654ce41da8a8…",200000000,21000000,null,null,"""0x0407af506d86bfa5e401099b2fc2…","""0xe7aa79b59cac06f9706d896a047f…","""0""","""0x29fbda675fa5a07b621c2c1a6e3f…","""452312848583266388373324160190…","""0x0000000000000000000000000000…","""0xcc467fdb41b5b5a4e23021cad69e…",null,177,"""0x43f8defe3e9286d152e91bb16a24…"
"""0x20c3035c92bdb4c461242571eeac…",22696105,1749823331,"""0x71ac12829d66ee73d8d95bff50b3…","""0x52322a25d9f59ea1765654554330…","""0x383f291819e6d54073bc9a648251…","""0x46adcbebc6be8ce551740c29c47c…","""0x9904ba90dde5696cda05c9e0dab5…","""0xa04c5bb938ca6fc46d95553abf0a…","""0x4b6c74f9e688cb39801f2112c14a…","""0xa11ee3ab75b40e88a0105e935d17…","""0x65a7ed542fb37fe237fdfbd

In [7]:
full_config.data.schema


Schema([('contract_address', String),
        ('block_number', Int64),
        ('block_timestamp', Int64),
        ('batch_inbox_slot', String),
        ('dispute_game_factory_slot', String),
        ('l1_cross_domain_messenger_slot', String),
        ('l1_erc721_bridge_slot', String),
        ('l1_standard_bridge_slot', String),
        ('optimism_mintable_erc20_factory_slot', String),
        ('optimism_portal_slot', String),
        ('start_block_slot', String),
        ('unsafe_block_signer_slot', String),
        ('version', String),
        ('basefee_scalar', Int64),
        ('batch_inbox', String),
        ('batcher_hash', String),
        ('blob_basefee_scalar', Int64),
        ('dispute_game_factory', String),
        ('eip1559_denominator', Int64),
        ('eip1559_elasticity', Int64),
        ('gas_limit', Int64),
        ('l1_cross_domain_messenger', String),
        ('l1_erc721_bridge', String),
        ('l1_standard_bridge', String),
        ('maximum_gas_limit', Int64),

In [40]:
from op_analytics.datasources.chainsmeta.systemconfig.endpoints import find_system_configs
from op_analytics.datasources.chainsmeta.systemconfig.systemconfig import SystemConfig
import requests

In [ ]:
# Create system config and get batch request
system_config = SystemConfig(system_config_proxy='0x229047fed2591dbec1eF1118d64F7aF3dB9EB290')
batch_request = system_config.rpc_batch()

print(f"\nBatch contains {len(batch_request)} requests")

In [ ]:
# Test with Ethereum RPC (since contracts are on L1)
ethereum_rpc = "https://ethereum-rpc.publicnode.com"
print(f"\nTesting with Ethereum RPC: {ethereum_rpc}")

response = requests.post(ethereum_rpc, json=batch_request, timeout=30)
response_data = response.json()

print(f"Response contains {len(response_data)} items")

In [ ]:
response_data

In [44]:
def debug_detailed_decoding():
    """Show detailed decoding analysis of all system config fields"""
    # Get chains
    # chains = find_system_configs()
    
    # # Test OP Mainnet (first chain)
    # test_chain = chains[0]  # OP Mainnet
    # print(f"Testing: {test_chain.identifier}")
    # print(f"System Config Proxy: {test_chain.system_config_proxy}")
    
    # Create system config and get batch request
    system_config = SystemConfig(system_config_proxy="0x158Fd5715F16Ac1F2Dc959A299B383aAaf9B59EB")
    
    # Test with Ethereum RPC (since contracts are on L1)
    ethereum_rpc = "https://ethereum-rpc.publicnode.com"
    print(f"\nTesting with Ethereum RPC: {ethereum_rpc}")
    
    try:
        # Get the raw response first
        batch_request = system_config.rpc_batch()
        response = requests.post(ethereum_rpc, json=batch_request, timeout=30)
        response_data = response.json()
        
        # Get the decoded result
        result = system_config.call_rpc(rpc_endpoint=ethereum_rpc)
        
        if result is None:
            print("❌ call_rpc returned None")
            return
        
        # Create a mapping of raw data for comparison
        data_map = {}
        for item in response_data:
            if 'error' not in item and item['id'] != 'block':
                data_map[item['id']] = item['result']
        
        print(f"\n" + "="*80)
        print("DETAILED DECODING ANALYSIS - ALL FIELDS")
        print("="*80)
        
        # Show all the slot values (these stay as hex strings)
        print("\n📍 SLOT VALUES (remain as hex strings):")
        print("-" * 50)
        slot_fields = [
            ('batch_inbox_slot', 'BATCH_INBOX_SLOT'),
            ('dispute_game_factory_slot', 'DISPUTE_GAME_FACTORY_SLOT'), 
            ('l1_cross_domain_messenger_slot', 'L1_CROSS_DOMAIN_MESSENGER_SLOT'),
            ('l1_erc721_bridge_slot', 'L1_ERC_721_BRIDGE_SLOT'),
            ('l1_standard_bridge_slot', 'L1_STANDARD_BRIDGE_SLOT'),
            ('optimism_mintable_erc20_factory_slot', 'OPTIMISM_MINTABLE_ERC20_FACTORY_SLOT'),
            ('optimism_portal_slot', 'OPTIMISM_PORTAL_SLOT'),
            ('start_block_slot', 'START_BLOCK_SLOT'),
            ('unsafe_block_signer_slot', 'UNSAFE_BLOCK_SIGNER_SLOT'),
        ]
        
        for field_name, data_key in slot_fields:
            raw_value = data_map.get(data_key, 'N/A')
            decoded_value = getattr(result, field_name, 'N/A')
            print(f"  {field_name}:")
            print(f"    Raw:     {raw_value}")
            print(f"    Decoded: {decoded_value}")
            print(f"    Type:    {type(decoded_value)}")
            print()
        
        print("\n🔢 INTEGER VALUES (decoded from hex):")
        print("-" * 50)
        integer_fields = [
            ('version', 'version', 'uint256'),
            ('basefee_scalar', 'basefeeScalar', 'uint32'),
            ('blob_basefee_scalar', 'blobbasefeeScalar', 'uint64'),
            ('eip1559_denominator', 'eip1559Denominator', 'uint32'),
            ('eip1559_elasticity', 'eip1559Elasticity', 'uint32'),
            ('gas_limit', 'gasLimit', 'uint64'),
            ('maximum_gas_limit', 'maximumGasLimit', 'uint64'),
            ('minimum_gas_limit', 'minimumGasLimit', 'uint64'),
            ('operator_fee_constant', 'operatorFeeConstant', 'uint64'),
            ('operator_fee_scalar', 'operatorFeeScalar', 'uint32'),
            ('overhead', 'overhead', 'uint256'),
            ('scalar', 'scalar', 'uint256'),
        ]
        
        for field_name, data_key, uint_type in integer_fields:
            raw_value = data_map.get(data_key, 'N/A')
            decoded_value = getattr(result, field_name, 'N/A')
            print(f"  {field_name} ({uint_type}):")
            print(f"    Raw:     {raw_value}")
            print(f"    Decoded: {decoded_value}")
            print(f"    Type:    {type(decoded_value)}")
            if isinstance(decoded_value, int):
                print(f"    Hex:     0x{decoded_value:x}")
                print(f"    Bits:    {decoded_value.bit_length()} bits")
            print()
        
        print("\n📍 ADDRESS VALUES (decoded from hex):")
        print("-" * 50)
        address_fields = [
            ('batch_inbox', 'batchInbox'),
            ('dispute_game_factory', 'disputeGameFactory'),
            ('l1_cross_domain_messenger', 'l1CrossDomainMessenger'),
            ('l1_erc721_bridge', 'l1ERC721Bridge'),
            ('l1_standard_bridge', 'l1StandardBridge'),
            ('optimism_mintable_erc20_factory', 'optimismMintableERC20Factory'),
            ('optimism_portal', 'optimismPortal'),
            ('owner', 'owner'),
            ('start_block', 'startBlock'),
            ('unsafe_block_signer', 'unsafeBlockSigner'),
        ]
        
        for field_name, data_key in address_fields:
            raw_value = data_map.get(data_key, 'N/A')
            decoded_value = getattr(result, field_name, 'N/A')
            print(f"  {field_name}:")
            print(f"    Raw:     {raw_value}")
            print(f"    Decoded: {decoded_value}")
            print(f"    Type:    {type(decoded_value)}")
            print()
        
        print("\n🔗 HASH VALUES:")
        print("-" * 50)
        hash_fields = [
            ('batcher_hash', 'batcherHash'),
        ]
        
        for field_name, data_key in hash_fields:
            raw_value = data_map.get(data_key, 'N/A')
            decoded_value = getattr(result, field_name, 'N/A')
            print(f"  {field_name}:")
            print(f"    Raw:     {raw_value}")
            print(f"    Decoded: {decoded_value}")
            print(f"    Type:    {type(decoded_value)}")
            print()
        
        print("\n🏷️  SPECIAL FIELDS:")
        print("-" * 50)
        print(f"  version_hex:")
        print(f"    Value:   {result.version_hex}")
        print(f"    Type:    {type(result.version_hex)}")
        print(f"    (This is hex(version)[2:] = hex({result.version})[2:])")
        print()
        
        print(f"  contract_address:")
        print(f"    Value:   {result.contract_address}")
        print(f"    Type:    {type(result.contract_address)}")
        print()
        
        print(f"  block_number:")
        print(f"    Value:   {result.block_number}")
        print(f"    Type:    {type(result.block_number)}")
        print()
        
        print(f"  block_timestamp:")
        print(f"    Value:   {result.block_timestamp}")
        print(f"    Type:    {type(result.block_timestamp)}")
        print()
        
        print("\n⚠️  PROBLEMATIC LARGE VALUES FOR POLARS:")
        print("-" * 50)
        large_values = [
            ('version', result.version),
            ('overhead', result.overhead),
            ('scalar', result.scalar),
        ]
        
        max_uint64 = 2**64 - 1
        for name, value in large_values:
            print(f"  {name}: {value}")
            print(f"    Type: {type(value)}")
            print(f"    Bit length: {value.bit_length()} bits")
            print(f"    Max UInt64: {max_uint64}")
            print(f"    Fits in UInt64: {value <= max_uint64}")
            print(f"    Fits in UInt32: {value <= 2**32 - 1}")
            if value > max_uint64:
                print(f"    ❌ TOO LARGE FOR POLARS UInt64!")
            print()
        
        print("\n📋 COMPLETE DICTIONARY OUTPUT:")
        print("-" * 50)
        sample_dict = result.to_dict()
        for key, value in sample_dict.items():
            value_str = str(value)
            if len(value_str) > 60:
                value_str = value_str[:60] + "..."
            print(f"  '{key}': {value_str} (type: {type(value)})")
            
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

In [ ]:
debug_detailed_decoding()